In [1]:
from jupyter_dash import JupyterDash
from dash import dcc, html, dash_table
from dash.dependencies import Input, Output
import pandas as pd
import dash_leaflet as dl
import plotly.express as px
from AnimalShelterClass import AnimalShelter

# Configure the necessary Python module imports
username = "aacuser"
password = "SNHU1234"
shelter = AnimalShelter(username, password, 'nv-desktop-services.apporto.com', 32433, 'graz', 'Rescue')

# Data Manipulation / Model
df = pd.DataFrame.from_records(shelter.read({}))
df.drop(columns=['_id'], inplace=True)

# Get unique animal types
unique_animal_types = shelter.get_unique_animal_types()
unique_animal_types = sorted(unique_animal_types)
unique_animal_types.insert(0, "Everything")

# Get list of column names
column_names = df.columns.tolist()

# Define column groups for demonstration
column_groups = {
    'Main Information': ['animal_id', 'animal_type', 'name', 'breed', 'color', 'date_of_birth', 'sex_upon_outcome'],  
    'All Columns': column_names
}

# Dashboard Layout / View
app = JupyterDash('SimpleExample', suppress_callback_exceptions=True)

app.layout = html.Div([
    html.Div(id='hidden-div', style={'display': 'none'}),
    html.Div(
        children=[
            html.Center(html.B(html.H1('SNHU CS-340 Dashboard Harley Reimels'))),
            html.Hr(),
            html.Div(
                id='buttonRow',
                style={'display': 'flex', 'justifyContent': 'center', 'marginBottom': '20px'},
                children=[
                    html.H3("Select Columns"),
                    dcc.Dropdown(
                        id='column-group-dropdown',
                        options=[{'label': group, 'value': group} for group in column_groups.keys()],
                        value='Main Information',  # Default value
                        style={'width': '50%', 'padding': '10px'}
                    ),
                    dcc.Checklist(
                        id='column-checkboxes',
                        style={'padding': '10px'}
                    ),
                    html.H3("Select Animal"),
                    dcc.Dropdown(
                        options=[
                            {'label': 'Show All', 'value': 'All'},
                            {'label': 'Water Rescue', 'value': 'Water Rescue'},
                            {'label': 'Wilderness Rescue', 'value': 'Wilderness Rescue'},
                            {'label': 'Disaster Rescue', 'value': 'Disaster Rescue'}
                        ],
                        value='All',  # Set a default value if needed
                        id='rescue-dropdown',  # Unique ID for this dropdown
                        style={'width': '250px', 'margin': 'auto', 'padding': '10px', 'borderRadius': '5px'}
                    )
                ]
            ),
            html.Div(
                id='datatable-container',
                style={'margin': '0 auto', 'width': '80%'},
                children=[
                    dash_table.DataTable(
                        id='datatable-id',
                        columns=[{"name": i, "id": i, "deletable": False, "selectable": True} for i in column_names],
                        data=df.to_dict('records'),
                        editable=False,
                        filter_action="native",
                        sort_action="native",
                        sort_mode="multi",
                        column_selectable=False,
                        row_selectable='single',
                        row_deletable=False,
                        selected_columns=[],
                        selected_rows=[],
                        style_data_conditional=[],
                        page_action="native",
                        page_current=0,
                        page_size=10,
                        style_table={
                            'overflowX': 'auto',  # Enable horizontal scrolling
                            'maxWidth': '100%'  # Ensure the table doesn't exceed the container width
                        },
                        style_header={
                            'backgroundColor': '#f8f9fa',
                            'fontWeight': 'bold'
                        },
                        style_cell={
                            'padding': '10px',
                            'textAlign': 'left',
                            'border': '1px solid #ddd',
                            'whiteSpace': 'normal',
                            'height': 'auto'
                        },
                        style_data={
                            'whiteSpace': 'normal',
                            'height': 'auto'
                        }
                    )
                ]
            ),
            html.Br(),
            html.Hr(),
            html.Div(
                id='map-and-chart-container',
                style={'display': 'flex', 'justifyContent': 'space-between', 'width': '100%'},
                children=[
                    html.Div(
                        id='map-container',
                        style={'width': '50%', 'height': '500px'},
                        children=[
                            html.Div(
                                id='map-border',
                                style={
                                    'border': '2px solid #007bff',  # Add a border around the map
                                    'height': '100%',  # Ensure the div takes the full height
                                    'width': '100%',  # Ensure the div takes the full width
                                    'position': 'relative'  # Ensure relative positioning for contained elements
                                },
                                children=[
                                    dl.Map(
                                        id='map-id',
                                        children=[
                                            dl.TileLayer()
                                        ],
                                        center=[56, 10],
                                        zoom=6,
                                        style={'height': '100%', 'width': '100%'}  # Ensure map fills the container
                                    )
                                ]
                            )
                        ]
                    ),
                    html.Div(
                        id='pie-chart-container',
                        style={'width': '45%', 'padding': '20px'},
                        children=[
                            dcc.Graph(id='breed-pie-chart')
                        ]
                    )
                ]
            )
        ]
    )
])

@app.callback(
    Output('column-checkboxes', 'children'),
    [Input('column-group-dropdown', 'value')]
)
def update_column_checkboxes(selected_group):
    columns = column_groups.get(selected_group, [])
    return dcc.Checklist(
        id='column-checkboxes-list',
        options=[{'label': col, 'value': col} for col in columns],
        value=columns,  # Default to showing all columns
        inline=True
    )


@app.callback(
    Output('datatable-id', 'columns'),
    [Input('column-checkboxes-list', 'value')]
)
def update_columns(selected_columns):
    return [{"name": col, "id": col, "deletable": False, "selectable": True} for col in selected_columns]


@app.callback(
    Output('datatable-id', "data"),
    [Input('rescue-dropdown', 'value')]
)
def on_click(rescue_type):  # Use 'rescue_type' based on the dropdown ID
    # Define the base queries
    water_query = {
        "animal_type": "Dog",
        "breed": {
            "$in": [
                "Labrador Retriever Mix",
                "Chesapeake Bay Retriever",
                "Newfoundland"
            ]
        },
        "sex_upon_outcome": "Intact Female",
        "age_upon_outcome_in_weeks": {
            "$gte": 26,
            "$lte": 156
        }
    }
    
    wilderness_query = {
        "animal_type": "Dog",
        "breed": {
            "$in": [
                "German Shepherd",
                "Alaskan Malamute",
                "Old English Sheepdog",
                "Siberian Husky",
                "Rottweiler"
            ]
        },
        "sex_upon_outcome": "Intact Male",
        "age_upon_outcome_in_weeks": {
            "$gte": 26,
            "$lte": 156
        }
    }
    
    disaster_query = {
        "animal_type": "Dog",
        "breed": {
            "$in": [
                "Doberman Pinscher",
                "German Shepherd",
                "Golden Retriever",
                "Bloodhound",
                "Rottweiler"
            ]
        },
        "sex_upon_outcome": "Intact Male",
        "age_upon_outcome_in_weeks": {
            "$gte": 20,
            "$lte": 300
        }
    }
    
    # Modify the query based on the selected filter type
    if rescue_type == 'Water Rescue':
        query = water_query
    elif rescue_type == 'Wilderness Rescue':
        query = wilderness_query
    elif rescue_type == 'Disaster Rescue':
        query = disaster_query
    else:  # Show All (default)
        query = {
            "animal_type": "Dog"
        }

    # Fetch data from MongoDB
    data = list(shelter.read(query))
    
    # Convert data to DataFrame and then to a format suitable for Dash DataTable
    df = pd.DataFrame(data)
    if '_id' in df.columns:
        df.drop(columns=['_id'], inplace=True)
    return df.to_dict('records')


@app.callback(
    Output('datatable-id', 'style_data_conditional'),
    [Input('datatable-id', 'selected_rows')]
)
def update_styles(selected_rows): # Used for highlighting rows
    if selected_rows is None:
        selected_rows = []

    return [{
        'if': {'row_index': i},
        'background_color': '#D2F3FF', # Highlights entire row and makes it pretty
        'font_weight': 'bold',
        'border': '1px solid #D2F3FF'
    } for i in selected_rows]


@app.callback(
    Output('map-id', "children"),
    [Input('datatable-id', "derived_virtual_data"),
     Input('datatable-id', "derived_virtual_selected_rows")]
)
def update_map(viewData, selected_rows):
    if not selected_rows or selected_rows[0] is None:
        # No selection or no rows selected
        return dl.Map([dl.TileLayer()], center=[56, 10], zoom=6, style={'height': '100%'}) # Default Map
    
    # Extract selected row's data
    selected_row_index = selected_rows[0]
    selected_data = viewData[selected_row_index]
    
    # Get latitude and longitude from the selected data
    lat = selected_data.get('location_lat', 56)  # Default to 56 if not available
    lon = selected_data.get('location_long', 10)  # Default to 10 if not available

    # Get name and description, defaulting if they are empty
    name = selected_data.get('name', 'No Name Provided')
    description = selected_data.get('breed', 'No Description Available')
    
    # If the name or description is an empty string, use the default
    name = name if name.strip() else 'No Name Provided'
    description = description if description.strip() else 'No Description Available'

    # Create the map with a marker
    map = dl.Map([
        dl.TileLayer(),
        dl.Marker(
            position=[lat, lon],
            children=[
                dl.Tooltip(selected_data.get('animal_type', 'No Animal Type')),
                dl.Popup([
                    html.H1(name),
                    html.P(description)
                ])
            ]
        )
    ], center=[lat, lon], zoom=8, style={'height': '100%'})

    return map

@app.callback(
    Output('breed-pie-chart', 'figure'),
    [Input('datatable-id', 'data')]
)
def update_pie_chart(data):
    if not data:
        return px.pie(title='No Data Available')
    
    # Convert data to DataFrame
    df = pd.DataFrame(data)
    
    # Create pie chart with Plotly
    fig = px.pie(df, names='breed', title='Breed Distribution')
    fig.update_layout(margin=dict(t=0, b=0, l=0, r=0))  # Remove margins for better fit
    return fig

app.run_server(debug=True)


Dash app running on http://127.0.0.1:11706/
